Duck typing - The emphasis is on what the object can do, as opposed to what it is. There is no check of the object's type and what it can do. Instead the code just runs, and if some object doesn't support the given operation, it errors. "If it walks like a duck, and talks like a duck, then it is a duck.

use `reprlib.repr` for safe representations of large or recursive structures. This way an ellipsis is used, instead of outputting thousands of lines of code.

\__repr__ should never error, because it is so important to debugging

## Protocols and Duck Typing

In OOP, a protocal is an informal interface, defined only in documentation and not in code.
An example would be the sequece protocol, which only requires the \__len__ and \__getitem__ methods.

A class that implements only those methods mentioned just above, should be identified as a sequence, because it appears to behave like one - not because it is explicilty stated to be one.

## Slicing

`slice` is a built-in type.

In [ ]:
dir(slice) # shows that a slice object has four attributes: start, step, stop, and indices

In [ ]:
help(slice.indices)

So what indices() does is show what Python does with missing or negative indices, and slices that are longer than the target sequence. Given a start, stop, step configuration, not is not "normal", or that is negative, this method normalizes them to fit within the bounds of the given sequence.

In [ ]:
slice(None, 10, 2).indices(5) # 'ABCDE'[:10:2] is the same as 'ABCDE'[0:5:2]
slice(-3, None, None).indices(5) # 'ABCDE'[-3:] is the same as 'ABCDE'[2:5:1]

In [ ]:
class Vector:
    # ommitting constructor
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, index):
        cls = type(self)
        if isistane(index, slice): # if the index is a slice object, return a new vector of that slice
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral): # if the index is some kind of integer, return that index
            # notice how an ABC, numbers.Integral, is used. This makes it more future proof and flexible.
            return self._components[index]
        else: #otherwise, error
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))

## Dynamic Attribute Access

\__getattr__ is invoked by the interpreter when looking up at attribute fails. If Python can't find x when `obj.x` is ran, and it can't be found in superclasses, then \__getattr__ is called.

Implementing a custom \__getattr__ method, such as for a multi dimensional vector class and getting the first few characters(xyz), can also require implementing a custom \__setattr__ method. This is because you don't want class attributes being set that \__getattr__ is depending on. Because once they're set, \__getattr__ will not be called, as it's only called if lookup fails. 

In [ ]:
class Vector:
    # ommitted other code
    
    def __setattr__(self, name, value):
        cls = type(self)
            if len(name) == 1:
                if name in cls.shortcut_names: # check if name in reserved attribtues used in __getattr__
                    error = 'readonly attribtue {attr_name!r}
                elif name.islower(): # don't allow any lower case letters in general
                    error = "can't set attirbutes 'a' to 'z' in {cls_name!r}"
                else: # otherwise bind error to empty string
                    error = ''
                if error: # if error refers to a non empty string, raise
                    msg = error.format(cls_name=cls.__name__, attr_name=name)
                    raise AttributeError(msg)
            super().__setattr__(name, value) #otherwise, call standard superclass method for setting attr

__very often__ if you implement `__getattr__`, you will need to implement `__setattr__`

__How functools.reduce works in Python__

reduce takes three arguments. A two argument function, an iterable, and an initiliaser

When you call reduce(fn, list), fn is applied to the first two elements of the list, creating r1. Then r1, and the third element of the list are inputted to fn, producing r2. And so forth until rN.

The initialiser is the value returned if the given iterable is empty, and is used as the first argument of the reducing loop. It should be the identity value of the operation. So 0 for +, |, ^, and 1 for *, &.

In [ ]:
# to get the hash of a multidimensional vector,
# follow the same logic of Vector2d, getting the XOR of each component's hash:

def __hash__(self):
#    hashes = (hash(x) for x in self._components)
    hashes = map(hash, self._components) # even better than the comprehension above
    # This could also call a lambda, but using an already build function, xor, is better
    return functools.reduce(operator.xor, hashes, 0)
    # This is a good example of map reduce. We mapped all components to their hashes, then made an xor of them into a signle value

Previously, the \__eq__ method was:

In [ ]:
def __eq__(self, other):
    return tuple(self) == tuple(other)

But this is ineffecient for a potentialyl huge mutlidimensional vector. Becuase it builds two seperate tuples, which means copying all of the components from both arguments. This could be a better implementation:

In [ ]:
def __eq__(self, other):
    if len(self) != len(other): # if unequal lengths, fail
        return False
    for a, b in zip(self, other): # the moment the component pairs don't match, fail.
        # as we're using zip, the len comparision is important zip stops producing values without warning, if a sequene is exhausted.
        # zip stops at the shortest operand.
        if a != b:
            return False
    return True

The above method can be made even more effecient using `all` function, which does the same aggregate computation of the for loop in one line:

In [ ]:
def __eq__(self, other):
    return len(self) == len(other) and all(a == b for a, b in zip(self, other))

__zip__

Somewhat suprisingly, zip will stop generating when one of the iterables is exhausted.
`itertools.longestzip` provides the opposite functionality. You give it a fillvalue as a parameter, which it uses so it can generate tuples until the last iterable is exhausted.

In [ ]:
import itertools
help(itertools.chain)

* If implementing a \__getattr__ method, you will usually also need a \__setattr__
* Look at how standard Python objects behave to emulate them and write good code

Protocols in Python are informal interfaces, unlike compiler enforced formal interfaces like in Java.

One only needs to implement the parts of a protocol that are relevant to the context.

We have high level languages to express our intentions at a higher level, and let the language worry about what low level operations are needed. Keep this in mind whenever coding. Is my code intuitive?